In [150]:
import Load_and_Save_Demo.UNet as UNet
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os


In [118]:
batchsize = 16

lr = 1e-3
epoch = 5

trainingdata = torch.rand([1000, 3, 32, 32])
validationdata = torch.rand([500, 3, 32, 32])
testingdata = torch.rand([500, 3, 32, 32])

traininglabel = torch.rand([1000, 1, 32, 32])
validationlabel = torch.rand([500, 1, 32, 32])
testinglabel = torch.rand([500, 1, 32, 32])

traincontent = []
for i in range(0, 1000):
    traindata = {'data': trainingdata[i], 'label': traininglabel[i]}
    traincontent.append(traindata)

valcontent = []
for i in range(0, 500):
    valdata = {'data': validationdata[i], 'label': validationlabel[i]}
    valcontent.append(valdata)

testcontent = []
for i in range(0, 500):
    testdata = {'data': testingdata[i], 'label': testinglabel[i]}
    testcontent.append(testdata)

'''
split the whole data into 1000, 500, 500 given that we have 2000 data points
'''

'\nsplit the whole data into 1000, 500, 500 given that we have 2000 data points\n'

In [119]:
model = UNet.UNet(n_channels=3, n_classes=1, bilinear=True)
model = model.to('cuda:0')
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
criterion = nn.BCEWithLogitsLoss().to('cuda:0')

#model, optimizer = amp.initialize(model, optimizer, opt_level='O1')
model = nn.DataParallel(model, device_ids=[0])
model.train()
'''
lock the model, no grad and weight updates
'''

'\nlock the model, no grad and weight updates\n'

In [120]:
for layer in model.state_dict():
    print(layer, '\n' ,model.state_dict()[layer].size(), '\n\n')

module.inc.double_conv.0.weight 
 torch.Size([64, 3, 3, 3]) 


module.inc.double_conv.0.bias 
 torch.Size([64]) 


module.inc.double_conv.1.weight 
 torch.Size([64]) 


module.inc.double_conv.1.bias 
 torch.Size([64]) 


module.inc.double_conv.1.running_mean 
 torch.Size([64]) 


module.inc.double_conv.1.running_var 
 torch.Size([64]) 


module.inc.double_conv.1.num_batches_tracked 
 torch.Size([]) 


module.inc.double_conv.3.weight 
 torch.Size([64, 64, 3, 3]) 


module.inc.double_conv.3.bias 
 torch.Size([64]) 


module.inc.double_conv.4.weight 
 torch.Size([64]) 


module.inc.double_conv.4.bias 
 torch.Size([64]) 


module.inc.double_conv.4.running_mean 
 torch.Size([64]) 


module.inc.double_conv.4.running_var 
 torch.Size([64]) 


module.inc.double_conv.4.num_batches_tracked 
 torch.Size([]) 


module.down1.maxpool_conv.1.double_conv.0.weight 
 torch.Size([128, 64, 3, 3]) 


module.down1.maxpool_conv.1.double_conv.0.bias 
 torch.Size([128]) 


module.down1.maxpool_conv.1.double_co

In [121]:
train_loader = DataLoader(traincontent, batch_size=batchsize, shuffle=True, num_workers=8, pin_memory=False, drop_last=True)
val_loader = DataLoader(valcontent, batch_size=batchsize, shuffle=True, num_workers=8, pin_memory=False, drop_last=True)
test_loader = DataLoader(testcontent, batch_size=batchsize, shuffle=True, num_workers=8, pin_memory=False, drop_last=True)
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

62
31
31


In [122]:
for epoch in range(epoch):
    loss_total = 0
    print(f'here in epoch {epoch}')
    for i, batch in tqdm(enumerate(train_loader)):
        data = batch['data'].to('cuda:0')
        label = batch['label'].to('cuda:0')
        if i == 0:
            print(data.size(), label.size())
        prediction = model(data)
        loss = criterion(prediction, label)
        loss_total += loss.item()
        loss.backward()
        optimizer.step()
    print(loss_total / len(train_loader))

    loss_total = 0
    for i, batch in tqdm(enumerate(val_loader)):
        data = batch['data'].to('cuda:0')
        label = batch['label'].to('cuda:0')
        prediction = model(data)
        loss = criterion(prediction, label)
        loss_total += loss.item()
    print(loss_total / len(val_loader))

here in epoch 0
torch.Size([16, 3, 32, 32]) torch.Size([16, 1, 32, 32])
0.7029016815846966
0.7009406282055762
here in epoch 1
torch.Size([16, 3, 32, 32]) torch.Size([16, 1, 32, 32])
0.6990690856210647
0.6980131960684254
here in epoch 2
torch.Size([16, 3, 32, 32]) torch.Size([16, 1, 32, 32])
0.696691644768561
0.6960666718021515
here in epoch 3
torch.Size([16, 3, 32, 32]) torch.Size([16, 1, 32, 32])
0.6950794102684144
0.6948411906919172
here in epoch 4
torch.Size([16, 3, 32, 32]) torch.Size([16, 1, 32, 32])
0.694183369798045
0.694092289094002


62it [00:03, 15.74it/s]
31it [00:00, 55.21it/s]
62it [00:03, 15.12it/s]
31it [00:00, 56.72it/s]
62it [00:03, 17.94it/s]
31it [00:00, 53.87it/s]
62it [00:03, 17.53it/s]
31it [00:00, 55.65it/s]
62it [00:03, 17.88it/s]
31it [00:00, 55.75it/s]


In [123]:
torch.save(model.state_dict(), os.path.join(r'D:\test', 'model.pth'))

In [148]:
savefile = r'D:\test\model.pth'



tmodel = UNet.UNet(n_channels=3, n_classes=1, bilinear=True)
dic = torch.load(savefile, map_location='cpu')
keys = list(dic.keys())

new_dic = tmodel.state_dict()
new_keys = list(new_dic.keys())

for i in range(128):
    new_dic[new_keys[i]] = dic[keys[i]]

tmodel.load_state_dict(new_dic)

tmodel = model.cuda()
tmodel.eval()

print(tmodel.state_dict())
# print(model.state_dict())

OrderedDict([('module.inc.double_conv.0.weight', tensor([[[[-0.2566,  0.3197, -0.4245],
          [-0.4410, -0.5493,  0.3639],
          [ 0.3354, -0.5719,  0.1108]],

         [[-0.2577, -0.2349, -0.4457],
          [-0.1002,  0.1448, -0.1268],
          [-0.0129, -0.3341, -0.2809]],

         [[-0.0195, -0.2420,  0.3043],
          [-0.4360, -0.3084,  0.3815],
          [ 0.0691,  0.4668, -0.4736]]],


        [[[-0.4769,  0.0416,  0.3190],
          [ 0.3252,  0.2202, -0.4240],
          [-0.2743, -0.4152,  0.2021]],

         [[ 0.4527,  0.0569, -0.0598],
          [-0.3068,  0.3641,  0.4440],
          [-0.4639,  0.4722,  0.0573]],

         [[ 0.4080, -0.3949,  0.2611],
          [-0.2024,  0.1373,  0.2244],
          [ 0.0429,  0.1548, -0.5135]]],


        [[[ 0.2055,  0.3917,  0.2529],
          [-0.1799, -0.3980, -0.3943],
          [-0.2602, -0.2399, -0.4067]],

         [[ 0.5569,  0.0185,  0.1235],
          [ 0.3024,  0.3953,  0.4583],
          [ 0.4147, -0.3393, -0.3395

In [149]:
with torch.no_grad():
    loss_total = 0
    for i, batch in tqdm(enumerate(test_loader)):
        data = batch['data'].to(device='cuda:0', dtype=torch.float32)
        label = batch['label'].to(device='cuda:0', dtype=torch.float32)
        prediction = tmodel(data)
        loss = criterion(prediction, label)
        loss_total += loss.item()
    print(loss_total / len(test_loader))



0it [00:00, ?it/s]
10it [00:00, 97.35it/s]
20it [00:00, 97.92it/s]
31it [00:00, 99.68it/s]

0.6940992320737531
